# Welcome to our ML project!

This is a quick exercise to help demonstrate your familiarity with RAG systems - one might say that this is a place where you can b**RAG** about your skills! 🤣

In this exercise, you will be asked to build a simple RAG system that answer some provided questions using the dataset provided. We expect this exercise to take 1-3 hours TOPS so use that to temper your approach to building this. We're not looking for reusable or production-level code - we're expressly looking for you to show us that you:

* can explore an unknown dataset
* can use an LLM (in this case, OpenAI's GPT-3) to build a simple RAG system

## The Dataset

You'll find the dataset in `content.csv`. It is a set of content about companies that has been scraped from the web. It contains the following columns:

* `company_id`: a unique identifier for the company (UUID)
* `company_name`: the name of the company
* `url`: the URL from which the content was scraped
* `chunk`: a chunk of the content that was scraped from the `url`
* `chunk_hash`: a hash of the chunk
* `chunk_id`: a unique identifier for the chunk of content
* `chunk_type`: the type of the chunk of content (e.g. `header`, `footer`)


Here's an example:

|company_id|company_name|url|chunk_type|chunk_hash|chunk|chunk_id|
|---|---|---|---|---|---|---|
|4c1fde18-8a40-4ee7-9c3c-19152c7d1ff8|Aboitiz Group|https://aboitiz.com/about-us/the-aboitiz-way/|head|d312f0c688076be80ee2e4af8a51c2f10cbb993a4a8de779cb4aa5545fe1051f|"<head>Aboitiz - The Aboitiz Way</head>"|be36e2f0-cd0b-42eb-b36d-c9403c2428be|

## Step 1: Explore the dataset

Here are some questions that we'd like you to answer about the dataset:

1. How many companies are in the dataset?
2. How many unique URLs are in the dataset?
3. What is the most common chunk type?
4. What is the distribution of chunk types by company?

In [14]:
%pip install pandas
%pip install matplotlib
%pip install openai

Note: you may need to restart the kernel to use updated packages.
  Using cached contourpy-1.2.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.50.0-cp311-cp311-macosx_10_9_universal2.whl.metadata (159 kB)
  Using cached kiwisolver-1.4.5-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.4 kB)
  Using cached pillow-10.2.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.7 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 25.5 MB/s eta 0:00:00a 0:00:01
Using cached contourpy-1.2.0-cp311-cp311-macosx_11_0_arm64.whl (243 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.50.0-cp311-cp311-macosx_10_9_universal2.whl (2.8 MB)
Using cached kiwisolver-1.4.5-cp311-cp311-macosx_11_0_arm64.whl (66 kB)
Using cached pillow-10.2.0-cp311-cp311-macosx_11_0_arm64.whl (3.3 MB)
Using cached pyparsing-3.1.2-py3-non

## Step 2: RAGtime!

Now that you're a little more familar with the dataset, let's build a simple RAG system that uses OpenAI to help answer some questions about the dataset. To reiterate, we don't expect you to add anything else to the environment to build this system - for example, you don't need to set up a database or anything like that. You can add any libraries you need to the environment, but we'd like you to use OpenAI for any and all tasks that require a language model (we'll send you a key to use).

Here is the question that we'd like you to answer via your RAG system:

1. What does the company Caravan Health do?